# Bunch Features.
Some time back I came across a video from Kaglle days where he was talking how easy it is to get a gold medal in a competition.Apart from the many interesting stuff he said, one thing that caught my attention feature engineering. Especially about creating bunch of features.

In this blog post, we will try out this method.

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from loguru import logger
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import matplotlib.pyplot as plt
from blog.model.regularisation import select_features, get_monotone_constraints
from blog.model.evaluation import show_model_results
from probatus.interpret import ShapModelInterpreter
from blog.data.lendingclub_dataset import LendingClubDataset
from blog.model.sampling import oot_split
from blog.model.regularisation import get_monotone_constraints
from blog.model.feature_transformations import get_simple_feature_transformation,create_bunch_feats
from blog.model.optimise import getObjective, tune_model
from blog.model.evaluation import get_segment_rocauc
from blog.model.target_transformations import get_sample_weights

In [2]:
dataset_name = "lendingclub"
target = "loan_status"
path = "../data/lc_accepted.csv"

initial_features = [
    "verification_status",
    "emp_title",
    "int_rate",
    "loan_amnt",
    "total_rec_int",
    "total_acc",
    "tot_cur_bal",
    "fico_range_low",
    "fico_range_high",
    "grade",
    "total_rev_hi_lim",
    "sub_grade",
    "initial_list_status",
    "purpose",
    "issue_d",
    "emp_length",
    "pub_rec_bankruptcies",
    "last_pymnt_amnt",
    "num_actv_bc_tl",
    "total_pymnt",
    "loan_status",
    "term",
    "home_ownership",
    "revol_util",
    "application_type",
    "addr_state",
    "inq_last_6mths",
    "pub_rec",
    "dti",
    "mort_acc",
    "revol_bal",
    "title",
    "annual_inc",
    "out_prncp",
    "open_acc",
    "installment",
    "home_ownership",
    "avg_cur_bal",
    "annual_inc",
    "num_tl_90g_dpd_24m"
]

# combine two lists without duplicates
initial_features = list(set(initial_features))
print(initial_features)
# ================== Read and clean the dataset.===========================================
logger.info(f"1.Read data")
lcd = LendingClubDataset()
X, y = lcd.get_data(path=path, use_cols=initial_features, dropna=False, sample=-1)
logger.info(f"\nTarget distribution:\n{y.value_counts(normalize=True)}")

# Split data into IT & OOT datasets.
data = X.copy()
data[target] = y

X_it, y_it, X_oot, y_oot = oot_split(
    df=data, split_date="Jun-2016", split_col="issue_d", target_col=target
)
# Original data dictionary
data_dict = {"xtrain": X_it, "ytrain": y_it, "xtest": X_oot, "ytest": y_oot}

# =================== Apply feature transformation =========================================
logger.info(f"2.Transform features")
data_dict = create_bunch_feats(data_dict=data_dict)
#data_dict = get_simple_feature_transformation(data_dict_org)


# =================== Feature Selection =========================================
logger.info(f"3.Starting feature selection.")
# Select the best features based on SHAPRFE CV
selected_features, fs_plot = select_features(data=data_dict, verbose=100)


# selected_features=['loan_amnt','int_rate','installment','grade','annual_inc','home_ownership',
#                    'emp_length','term','addr_state', 
#                    'fico_range_low', 'fico_range_high']

# selected_features = [
#     "emp_length",
#     "addr_state",
#     "revol_util",
#     "revol_bal",
#     "term",
#     "num_actv_bc_tl",
#     "total_acc",
#     "application_type",
#     "purpose",
#     "grade",
#     'dti',
#     "home_ownership",
#     "annual_inc",
#     "num_tl_90g_dpd_24m"
# ]
# logger.info(f"Final features :  {selected_features}")
# # Update the data dictionary with the selected features.
# data_dict["xtrain"] = data_dict["xtrain"][selected_features]
# data_dict["xtest"] = data_dict["xtest"][selected_features]

# model_params = {
#     "objective": "binary",
#     "metric": "binary_logloss",
#     "verbosity": -1,
#     "boosting_type": "gbdt",
#     "feature_pre_filter": False,
#     "lambda_l1": 3.221919143923753e-08,
#     "lambda_l2": 0.0016740960905730054,
#     "num_leaves": 66,
#     "feature_fraction": 0.6,
#     "bagging_fraction": 1.0,
#     "bagging_freq": 0,
#     "min_child_samples": 20,
# }

# # model_params = tune_model(data_dict["xtrain"], data_dict["ytrain"], "rocauc")

# # =================== Train Model =========================================
# logger.info(f"5.Train Model ")
# logger.info(f"Creating model with params : {model_params}")
# mono_lgb = lgb.LGBMClassifier(**model_params)
# sample_weight = get_sample_weights(data_dict)
# mono_model = show_model_results(data=data_dict, model=mono_lgb,sample_weight=sample_weight)

2022-04-23 22:18:55.505 | INFO     | __main__:<module>:52 - 1.Read data
2022-04-23 22:18:55.516 | INFO     | blog.data.lendingclub_dataset:_read_data:24 - Reading data from path ../data/lc_accepted.csv


['total_rec_int', 'installment', 'addr_state', 'emp_title', 'loan_amnt', 'last_pymnt_amnt', 'purpose', 'home_ownership', 'num_actv_bc_tl', 'fico_range_high', 'verification_status', 'num_tl_90g_dpd_24m', 'tot_cur_bal', 'total_rev_hi_lim', 'title', 'avg_cur_bal', 'total_acc', 'initial_list_status', 'revol_util', 'issue_d', 'sub_grade', 'mort_acc', 'out_prncp', 'inq_last_6mths', 'revol_bal', 'pub_rec_bankruptcies', 'int_rate', 'total_pymnt', 'pub_rec', 'dti', 'loan_status', 'grade', 'annual_inc', 'fico_range_low', 'emp_length', 'application_type', 'open_acc', 'term']


2022-04-23 22:19:20.112 | INFO     | blog.data.lendingclub_dataset:_read_data:28 - Read data with shape (2260701, 38)
2022-04-23 22:19:24.435 | INFO     | blog.data.lendingclub_dataset:_process_data:73 - Creating target data.
2022-04-23 22:19:35.284 | INFO     | blog.data.lendingclub_dataset:get_data:118 - Shape of training data X :(1251248, 39), y : (1251248,).
2022-04-23 22:19:35.302 | INFO     | __main__:<module>:55 - 
Target distribution:
0    0.797347
1    0.202653
Name: loan_status, dtype: float64
2022-04-23 22:19:36.597 | INFO     | blog.model.sampling:oot_split:63 - IT data info 
 : {'name': 'IT', 'X_shape': (894832, 40), 'y_shape': (894832,), 'default_rate': 19.289877876517604, 'pct_of_data': 71.5, 'earliest_disbursed_date': Timestamp('2012-08-01 00:00:00'), 'latest_disbursed_date': Timestamp('2016-05-01 00:00:00'), 'default_count': 172612}
2022-04-23 22:19:36.599 | INFO     | blog.model.sampling:oot_split:64 - OOT data info 
 : {'name': 'OOT', 'X_shape': (356416, 40), 'y_shap

[LightGBM] [Warning] lambda_l1 is set=3.221919143923753e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.221919143923753e-08
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] lambda_l2 is set=0.0016740960905730054, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0016740960905730054
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] lambda_l1 is set=3.221919143923753e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.221919143923753e-08
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fract